In [1]:
import pandas as pd
import numpy as np
from copy import deepcopy
import time

In [2]:
pd.set_option('max_rows',2000)
pd.set_option('max_columns',2000)
np.set_printoptions()

In [3]:
df = pd.read_stata('/scratch/sv1239/projects/mlcs/raw/Votelevel_stuffjan2013.dta')
# df = pd.read_stata('Votelevel_stuffjan2013.dta')

In [4]:
df.shape

(56403, 868)

In [6]:
df[df.casenum==12954]

,casenum,fileid,year,month,day,cite,vol,beginpg,endopin,endpage,docnum,multdoc,method,circuit,state,district,origin,source,distjudg,applfrom,adminrev,priorpub,opinstat,treat,classact,crossapp,majvotes,dissent,concur,counsel1,counsel2,amicus,sanction,initiate,interven,numappel,appnatpr,appbus,appnonp,appfed,appsubst,appstate,appfiduc,ap_stid,genapel1,bank_ap1,genapel2,bank_ap2,appel1,appel2,realapp,numresp,r_natpr,r_bus,r_nonp,r_fed,r_subst,r_state,r_fiduc,r_stid,respond1,respond2,realresp,genresp1,genresp2,bank_r1,bank_r2,const1,const2,usc1,usc1sect,usc2,usc2sect,civproc1,civproc2,crmproc1,crmproc2,juris,statecl,standing,mootness,exhaust,timely,immunity,frivol,polquest,oththres,late,frivapp,othappth,constit,fedlaw,procedur,typeiss,prejud,insane,improper,juryinst,othjury,deathpen,sentence,indict,confess,search,othadmis,plea,counsel,rtcouns,suffic,indigent,entrap,procdis,othcrim,dueproc,execord,stpolicy,weightev,pretrial,trialpro,post_trl,attyfee,judgdisc,altdisp,injunct,summary,fedvst,foreign,int_law,st_v_st,discover,othcivil,subevid,denovo,erron,capric,abusedis,judrev,genstand,notice,alj,agen_acq,freeinfo,comment,record,diverse,whlaws,casetyp1,geniss,direct1,casetyp2,direct2,habeas,decuncon,unan,signed,ids,codej2,j2vote1,j2vote2,j2maj1,j2maj2,codej3,j3vote1,j3vote2,j3maj1,j3maj2,codej4,j4vote1,j4vote2,j4maj1,j4maj2,codej5,j5vote1,j5vote2,j5maj1,j5maj2,codej6,j6vote1,j6vote2,j6maj1,j6maj2,codej7,j7vote1,j7vote2,j7maj1,j7maj2,codej8,j8vote1,j8vote2,j8maj1,j8maj2,codej9,j9vote1,j9vote2,j9maj1,j9maj2,codej10,j10vote1,j10vote2,j10maj1,j10maj2,codej11,j11vote1,j11vote2,j11maj1,j11maj2,codej12,j12vote1,j12vote2,j12maj1,j12maj2,codej13,j13vote1,j13vote2,j13maj1,j13maj2,codej14,j14vote1,j14vote2,j14maj1,j14maj2,codej15,j15vote1,j15vote2,j15maj1,j15maj2,pagelgth,j16maj1,j16vote1,citevol,decision_date,alst_code,nofirmap,noattyap,aatty_first_name,aatty_last_name,afirm_name,amicusapp,a_groups,afirst_listed_amicus_group,r_appsubst,rlst_code,nofirmrp,noattyrp,ratty_first_name,ratty_last_name,amicusresp,r_groups,rname_of_first_listed_amicus_gro,isspty,dissmer,concmer1,concmer2,disspro,concpro1,concpro2,fedlaw2,keytotal,threshgen,turnonthresh,rfirm_namew,decisiondatenew2,dropflag,j1name,j1party,j1appres,j1distjudge,j1sample,j1score,moved,j2name,j2party,j2appres,j2distjudge,j2sample,j2score,j3name,j3party,j3appres,j3distjudge,j3sample,j3score,sumappress,divided,oldj1party,oldj2party,oldj3party,sumparty,divided1,sumpartyappt,dividedA,districtjudgeinpanel,nomoversinpanel,concurdummy,dissentdummy,unanimous,quarter,fromdistrictcourt,typeiss2,geniss2,geniss3,yearquarter,decade2,decade,distance,quartertoelect,semiannumtoelect,lastquarter,sixmonthsbefore,sixmonthsafter,lengthopin,Wopinionlenght,totalcites,negativecites,propneg,Wtotalcites,Wlengthopin,lastthreequarter,dissmerdummy,dissprodummy,disspromerdummy,disspro0merdummy,diss0promerdummy,diss0pro0merdummy,recent,type1,type2,type3,type4,type5,type6,type7,affirmdummy,reversedummy,initiate1,initiate2,initiate3,initiate4,initiate5,concmerdummy,concprodummy,concpromerdummy,concpro0merdummy,conc0promerdummy,conc0pro0merdummy,month911_f4,month911_f3,month911_f2,month911_f1,month911_t0,month911_t1,month911_t2,month911_t3,month911_t4,month911_t5,month911_t6,month911_3mon,last3,ww2,korean,vietnam,gulf,afghan,war,clinton,caseload,president,senate,president_f1,president_f2,senate_f1,senate_f2,electoralpct,popularpct,var10,var11,repres,rsen,repres_f1,rsen_f1,repres_f2,rsen_f2,j,codej1,dissentvote,concurauthor,concurjoin,pname,id,pres,mona,yeara,monl,yearl,yearb,yeard,pleft,left,party,city,elevate,gender,race,name,appres,pleftsum,boom,crossa,crossl,mrespty,prespty,dummy,religion,csb,ba,bast,bapp,ls,lsst,jdpp,graddeg1,graddeg2,graddeg3,pfedjdge,pfjdget,pssc,pslc,plocct,poutsjge,psjdget,pausa,pusa,psgo,psg,pago,paag,pag,pcc,psp,pspros,pmag,pbj,pcab,pcabdept,pscab,pscabdpt,pindreg1,preg1,pindreg2,preg2,pindreg3,preg3,pindreg4,preg4,pindreg5,preg5,phouse,psenate,pgov,pssenate,pshouse,pmayor,pccoun,pccom,pda,plother

In [5]:
df.head()

,casenum,fileid,year,month,day,cite,vol,beginpg,endopin,endpage,docnum,multdoc,method,circuit,state,district,origin,source,distjudg,applfrom,adminrev,priorpub,opinstat,treat,classact,crossapp,majvotes,dissent,concur,counsel1,counsel2,amicus,sanction,initiate,interven,numappel,appnatpr,appbus,appnonp,appfed,appsubst,appstate,appfiduc,ap_stid,genapel1,bank_ap1,genapel2,bank_ap2,appel1,appel2,realapp,numresp,r_natpr,r_bus,r_nonp,r_fed,r_subst,r_state,r_fiduc,r_stid,respond1,respond2,realresp,genresp1,genresp2,bank_r1,bank_r2,const1,const2,usc1,usc1sect,usc2,usc2sect,civproc1,civproc2,crmproc1,crmproc2,juris,statecl,standing,mootness,exhaust,timely,immunity,frivol,polquest,oththres,late,frivapp,othappth,constit,fedlaw,procedur,typeiss,prejud,insane,improper,juryinst,othjury,deathpen,sentence,indict,confess,search,othadmis,plea,counsel,rtcouns,suffic,indigent,entrap,procdis,othcrim,dueproc,execord,stpolicy,weightev,pretrial,trialpro,post_trl,attyfee,judgdisc,altdisp,injunct,summary,fedvst,foreign,int_law,st_v_st,discover,othcivil,subevid,denovo,erron,capric,abusedis,judrev,genstand,notice,alj,agen_acq,freeinfo,comment,record,diverse,whlaws,casetyp1,geniss,direct1,casetyp2,direct2,habeas,decuncon,unan,signed,ids,codej2,j2vote1,j2vote2,j2maj1,j2maj2,codej3,j3vote1,j3vote2,j3maj1,j3maj2,codej4,j4vote1,j4vote2,j4maj1,j4maj2,codej5,j5vote1,j5vote2,j5maj1,j5maj2,codej6,j6vote1,j6vote2,j6maj1,j6maj2,codej7,j7vote1,j7vote2,j7maj1,j7maj2,codej8,j8vote1,j8vote2,j8maj1,j8maj2,codej9,j9vote1,j9vote2,j9maj1,j9maj2,codej10,j10vote1,j10vote2,j10maj1,j10maj2,codej11,j11vote1,j11vote2,j11maj1,j11maj2,codej12,j12vote1,j12vote2,j12maj1,j12maj2,codej13,j13vote1,j13vote2,j13maj1,j13maj2,codej14,j14vote1,j14vote2,j14maj1,j14maj2,codej15,j15vote1,j15vote2,j15maj1,j15maj2,pagelgth,j16maj1,j16vote1,citevol,decision_date,alst_code,nofirmap,noattyap,aatty_first_name,aatty_last_name,afirm_name,amicusapp,a_groups,afirst_listed_amicus_group,r_appsubst,rlst_code,nofirmrp,noattyrp,ratty_first_name,ratty_last_name,amicusresp,r_groups,rname_of_first_listed_amicus_gro,isspty,dissmer,concmer1,concmer2,disspro,concpro1,concpro2,fedlaw2,keytotal,threshgen,turnonthresh,rfirm_namew,decisiondatenew2,dropflag,j1name,j1party,j1appres,j1distjudge,j1sample,j1score,moved,j2name,j2party,j2appres,j2distjudge,j2sample,j2score,j3name,j3party,j3appres,j3distjudge,j3sample,j3score,sumappress,divided,oldj1party,oldj2party,oldj3party,sumparty,divided1,sumpartyappt,dividedA,districtjudgeinpanel,nomoversinpanel,concurdummy,dissentdummy,unanimous,quarter,fromdistrictcourt,typeiss2,geniss2,geniss3,yearquarter,decade2,decade,distance,quartertoelect,semiannumtoelect,lastquarter,sixmonthsbefore,sixmonthsafter,lengthopin,Wopinionlenght,totalcites,negativecites,propneg,Wtotalcites,Wlengthopin,lastthreequarter,dissmerdummy,dissprodummy,disspromerdummy,disspro0merdummy,diss0promerdummy,diss0pro0merdummy,recent,type1,type2,type3,type4,type5,type6,type7,affirmdummy,reversedummy,initiate1,initiate2,initiate3,initiate4,initiate5,concmerdummy,concprodummy,concpromerdummy,concpro0merdummy,conc0promerdummy,conc0pro0merdummy,month911_f4,month911_f3,month911_f2,month911_f1,month911_t0,month911_t1,month911_t2,month911_t3,month911_t4,month911_t5,month911_t6,month911_3mon,last3,ww2,korean,vietnam,gulf,afghan,war,clinton,caseload,president,senate,president_f1,president_f2,senate_f1,senate_f2,electoralpct,popularpct,var10,var11,repres,rsen,repres_f1,rsen_f1,repres_f2,rsen_f2,j,codej1,dissentvote,concurauthor,concurjoin,pname,id,pres,mona,yeara,monl,yearl,yearb,yeard,pleft,left,party,city,elevate,gender,race,name,appres,pleftsum,boom,crossa,crossl,mrespty,prespty,dummy,religion,csb,ba,bast,bapp,ls,lsst,jdpp,graddeg1,graddeg2,graddeg3,pfedjdge,pfjdget,pssc,pslc,plocct,poutsjge,psjdget,pausa,pusa,psgo,psg,pago,paag,pag,pcc,psp,pspros,pmag,pbj,pcab,pcabdept,pscab,pscabdpt,pindreg1,preg1,pindreg2,preg2,pindreg3,preg3,pindreg4,preg4,pindreg5,preg5,phouse,psenate,pgov,pssenate,pshouse,pmayor,pccoun,pccom,pda,plother

In [7]:
# columns to be removed, casenum or codej1... or j2vote... or j2maj... will be removed later
del_cols = ['fileid','cite','vol','beginpg','endopin','endpage','docnum','priorpub','_merge','year',
            'circuit','pseatno','decision_date','aatty_first_name','aatty_last_name','afirm_name',
            'ratty_first_name','ratty_last_name','rname_of_first_listed_amicus_gro','rfirm_namew','decisiondatenew2',
           'j1name','j2name','j3name','quartertoelect','pname','seatno','success','lsuc','ls1','ls2','ls3','lp',
            'lp2','lp3','sseatno','congress','congreso','afirst_listed_amicus_group','yearquarter','name','Name','State','j',
            'codej4','j4vote1','j4vote2','j4maj1','j4maj2','codej5','j5vote1','j5vote2','j5maj1','j5maj2',
            'codej6','j6vote1','j6vote2','j6maj1','j6maj2','codej7','j7vote1','j7vote2','j7maj1','j7maj2',
            'codej8','j8vote1','j8vote2','j8maj1','j8maj2','codej9','j9vote1','j9vote2','j9maj1','j9maj2',
            'codej10','j10vote1','j10vote2','j10maj1','j10maj2','codej11','j11vote1','j11vote2','j11maj1','j11maj2',
            'codej12','j12vote1','j12vote2','j12maj1','j12maj2','codej13','j13vote1','j13vote2','j13maj1','j13maj2',
            'codej14','j14vote1','j14vote2','j14maj1','j14maj2','codej15','j15vote1','j15vote2','j15maj1','j15maj2','j16maj1','j16vote1']
#
# to_dummies = ['month','day','method','state','district','origin','source','distjudg',
#               'applfrom','adminrev','opinstat','treat','classact','crossapp','counsel1','counsel2','sanction',
#               'initiate','numappel','appnatpr','appnatpr','appbus','appnonp','appfed','appsubst','appstate',
#               'appfiduc','ap_stid','genapel1','bank_ap1','genapel2','bank_ap2','appel1','appel2',]

In [8]:
df.drop(labels=del_cols,axis=1,inplace=True)

In [75]:
#df.shape

(56403, 762)

In [9]:
moredropcolumns=df.columns.tolist() # .tolist?
for i in moredropcolumns:
    if len(pd.unique(df[i]))==1:
        df.drop(labels=i,axis=1,inplace=True)

In [77]:
#df.shape

(56403, 743)

In [11]:
caseList=pd.unique(df['casenum'])
caseList=caseList[pd.notnull(caseList)].tolist()

In [12]:
caseList

[12954.0,
 14244.0,
 4900.0,
 6449.0,
 579.0,
 12752.0,
 21722.0,
 9045.0,
 417.0,
 14835.0,
 11157.0,
 4109.0,
 17315.0,
 7606.0,
 4992.0,
 4365.0,
 6911.0,
 2698.0,
 21502.0,
 1524.0,
 429.0,
 8869.0,
 13032.0,
 14615.0,
 8612.0,
 12204.0,
 5023.0,
 1610.0,
 7697.0,
 17464.0,
 93.0,
 16674.0,
 14955.0,
 11632.0,
 21.0,
 9799.0,
 17873.0,
 12857.0,
 4869.0,
 2788.0,
 9553.0,
 3736.0,
 8610.0,
 20813.0,
 4437.0,
 20717.0,
 1044.0,
 3825.0,
 3568.0,
 20931.0,
 9751.0,
 21660.0,
 12925.0,
 15674.0,
 7058.0,
 11143.0,
 3833.0,
 2628.0,
 2711.0,
 14644.0,
 12916.0,
 14486.0,
 21497.0,
 17072.0,
 21395.0,
 22015.0,
 5860.0,
 11393.0,
 15636.0,
 4478.0,
 3356.0,
 8387.0,
 2489.0,
 9809.0,
 1493.0,
 16299.0,
 456.0,
 20051.0,
 9283.0,
 9693.0,
 5134.0,
 14079.0,
 11978.0,
 3036.0,
 2362.0,
 9139.0,
 18024.0,
 3910.0,
 21027.0,
 7688.0,
 15447.0,
 6638.0,
 2814.0,
 3582.0,
 16548.0,
 14527.0,
 5936.0,
 14833.0,
 13712.0,
 2702.0,
 6735.0,
 15531.0,
 9322.0,
 21629.0,
 14468.0,
 8670.0,
 16305.

In [10]:
## I use case id to find the record of each vote of the same case, caseList save each unique case id for one time. 
## There are 18000+ unique cases.

caseList=[]
for i in range(df.shape[0]):
    if pd.notnull(df.ix[i]["casenum"]) & (df.ix[i]["casenum"] not in caseList):
        caseList.append(df.ix[i]["casenum"])
print len(caseList)

KeyboardInterrupt: 

In [13]:
newframe=pd.DataFrame()                ##  the rearrange of the original data
output=[]
case=caseList[0]
subtest=df[df.casenum==case].reset_index(drop=True)

In [17]:
subtest.codej1

0    307
1    307
2    NaN
Name: codej1, dtype: float64

In [14]:
subtest

,casenum,month,day,multdoc,method,state,district,origin,source,distjudg,applfrom,adminrev,opinstat,treat,classact,crossapp,majvotes,dissent,concur,counsel1,counsel2,amicus,sanction,initiate,interven,numappel,appnatpr,appbus,appnonp,appfed,appsubst,appstate,appfiduc,ap_stid,genapel1,bank_ap1,genapel2,bank_ap2,appel1,appel2,realapp,numresp,r_natpr,r_bus,r_nonp,r_fed,r_subst,r_state,r_fiduc,r_stid,respond1,respond2,realresp,genresp1,genresp2,bank_r1,bank_r2,const1,const2,usc1,usc1sect,usc2,usc2sect,civproc1,civproc2,crmproc1,crmproc2,juris,statecl,standing,mootness,exhaust,timely,immunity,frivol,polquest,oththres,late,frivapp,othappth,constit,fedlaw,procedur,typeiss,prejud,insane,improper,juryinst,othjury,deathpen,sentence,indict,confess,search,othadmis,plea,counsel,rtcouns,suffic,indigent,entrap,procdis,othcrim,dueproc,execord,stpolicy,weightev,pretrial,trialpro,post_trl,attyfee,judgdisc,altdisp,injunct,summary,fedvst,foreign,int_law,st_v_st,discover,othcivil,subevid,denovo,erron,capric,abusedis,judrev,genstand,notice,alj,agen_acq,freeinfo,comment,record,diverse,whlaws,casetyp1,geniss,direct1,casetyp2,direct2,habeas,decuncon,unan,signed,ids,codej2,j2vote1,j2vote2,j2maj1,j2maj2,codej3,j3vote1,j3vote2,j3maj1,j3maj2,pagelgth,citevol,alst_code,nofirmap,noattyap,amicusapp,a_groups,r_appsubst,rlst_code,nofirmrp,noattyrp,amicusresp,r_groups,isspty,dissmer,concmer1,concmer2,disspro,concpro1,concpro2,fedlaw2,keytotal,threshgen,turnonthresh,dropflag,j1party,j1appres,j1distjudge,j1sample,j1score,moved,j2party,j2appres,j2distjudge,j2sample,j2score,j3party,j3appres,j3distjudge,j3sample,j3score,sumappress,divided,oldj1party,oldj2party,oldj3party,sumparty,divided1,sumpartyappt,dividedA,districtjudgeinpanel,nomoversinpanel,concurdummy,dissentdummy,unanimous,quarter,fromdistrictcourt,typeiss2,geniss2,geniss3,decade2,decade,distance,semiannumtoelect,lastquarter,sixmonthsbefore,sixmonthsafter,lengthopin,Wopinionlenght,totalcites,negativecites,propneg,Wtotalcites,Wlengthopin,lastthreequarter,dissmerdummy,dissprodummy,disspromerdummy,disspro0merdummy,diss0promerdummy,diss0pro0merdummy,recent,type1,type2,type3,type4,type5,type6,type7,affirmdummy,reversedummy,initiate1,initiate2,initiate3,initiate4,initiate5,concmerdummy,concprodummy,concpromerdummy,concpro0merdummy,conc0promerdummy,conc0pro0merdummy,month911_f4,month911_f3,month911_f2,month911_f1,month911_t0,month911_t1,month911_t2,month911_t3,month911_t4,month911_t5,month911_t6,month911_3mon,last3,ww2,korean,vietnam,gulf,afghan,war,clinton,caseload,president,senate,president_f1,president_f2,senate_f1,senate_f2,electoralpct,popularpct,repres,rsen,repres_f1,rsen_f1,repres_f2,rsen_f2,codej1,dissentvote,concurauthor,concurjoin,id,pres,mona,yeara,monl,yearl,yearb,yeard,pleft,left,party,city,elevate,gender,race,appres,pleftsum,boom,crossa,crossl,mrespty,prespty,dummy,religion,csb,ba,bast,bapp,ls,lsst,jdpp,graddeg1,graddeg2,graddeg3,pfedjdge,pfjdget,pssc,pslc,plocct,poutsjge,psjdget,pausa,pusa,psgo,psg,pago,paag,pag,pcc,psp,pspros,pmag,pbj,pcab,pcabdept,pscab,pscabdpt,pindreg1,preg1,pindreg2,preg2,pindreg3,preg3,pindreg4,preg4,pindreg5,preg5,phouse,psenate,pgov,pssenate,pshouse,pmayor,pccoun,pccom,pda,plother,plotherl,pgovt,plawprof,pprivate,prevpos,pmagis,pcomct,pdat,pada,pbank,pterr,pcustom,statecab,age,amon,ayear,___l,___j,___char,pos1,pos1yrs,pos2,pos2yrs,pos3,pos3yrs,pos4,pos4yrs,pos5,pos5yrs,pos6,pos6yrs,pos7,pos7yrs,pos8,pos8yrs,pos9,pos9yrs,pos10,pos10yrs,pos11,pos11yrs,pos12,pos12yrs,type,pred,recdatem,recdatey,reccon,extra5,assets,liable,aba,add1,abamin,seatno2,count,seatno3,pseatno3,sseatno3,pseatno2,sseatno2,sids,tids,samon,sayear,tamon,tayear,dsen,dhouse,rhouse,dal,dak,daz,dar,dca,dco,dct,dde,dfl,dga,dhi,did,dil,din,dio,dks,dky,dla,dme,dmd,dma,dmi,dmn,dms,dmo,dmt,dne,dnv,dnh,dnj,dnm,dny,dnc,dnd,doh,dok,dor,dpa,dri,dsc,dsd,dtn,dtx,dut,dvt,dva,dwa,dwv,dwi,dwy,source1,source2,courtesy,appage,pname2,pname3,dcother,uporoutt,senmaj,senmin,yeari,yearc,unity,e1,e2,e3,e4,e5,e6,congresi,unityi,hdem,hrep

In [15]:
num=subtest.shape[0]

In [18]:
## on my computer, about 0.5 second per case
## there will be 6 rows for each case. codej1 correspond to primary judge 
newframe=pd.DataFrame()                ##  the rearrange of the original data
output=[]                           ##   the corresponding alignment of judge 1 and judge 2, yes =1, no = -1
for case in caseList[:10]:    
    subtest=df[df.casenum==case].reset_index(drop=True)  ## 'subtest' only take the records that have a specific case id
    num=subtest.shape[0]                                 ## num will be 3, because usally there are 3 records for each case 
    for j in range(num):
        if pd.notnull(subtest.ix[j].codej1):
            j1=subtest.ix[j].codej1
        if pd.notnull(subtest.ix[j].codej2):
            j2=subtest.ix[j].codej2
        if pd.notnull(subtest.ix[j].codej3):
            j3=subtest.ix[j].codej3
    if subtest.ix[0].majvotes == 3:
        j1j2=1
        j1j3=1
        j2j3=1
    else:
        if subtest.ix[0].direct1==subtest.ix[0].j2vote1:
            j1j2=1
        else:
            j1j2=-1                
        if subtest.ix[0].direct1==subtest.ix[0].j3vote1:
            j1j3=1
        else:
            j1j3=-1
        if subtest.ix[0].j2vote1==subtest.ix[0].j3vote1:
            j2j3=1
        else:
            j2j3=-1
    for j in range(num):
        copytest=deepcopy(subtest.ix[j])
        if copytest.ids==j1:
            copytest.codej1=j1
            newframe=newframe.append(copytest)
            output.append(j1j2)
            copytest.codej2=j3
            copytest.codej3=j2
            newframe=newframe.append(copytest)
            output.append(j1j3)
        if copytest.ids==j2:
            copytest.codej1=j2
            copytest.codej2=j1
            newframe=newframe.append(copytest)
            output.append(j1j2)
            copytest.codej2=j3
            copytest.codej3=j1
            newframe=newframe.append(copytest)
            output.append(j2j3)
        if copytest.ids==j3:
            copytest.codej1=j3
            copytest.codej3=j1
            newframe=newframe.append(copytest)
            output.append(j2j3)
            copytest.codej2=j1
            copytest.codej3=j2
            newframe=newframe.append(copytest)
            output.append(j1j3)
            
newframe=newframe.reset_index()              ## need to reset the index, otherwise will all be 0
newframe=newframe.drop('index',1)  

assert newframe.shape[0]==len(output)
print newframe.shape

(60, 743)


In [22]:
newframe.ix[:5]

,Ads3,ElecYear_AndPrior,F1Ads3,F2Ads3,L1Ads3,L2Ads3,L3Ads3,L4Ads3,L5Ads3,Wlengthopin,Wopinionlenght,Wtotalcites,_Icircuit_1,_Icircuit_10,_Icircuit_11,_Icircuit_2,_Icircuit_3,_Icircuit_4,_Icircuit_5,_Icircuit_6,_Icircuit_7,_Icircuit_8,_Icircuit_9,_Igeniss2_1,_Igeniss2_2,_Igeniss2_3,_Igeniss2_6,_Igeniss2_7,_Ilastquart_1,_Iquarter_2,_Iquarter_3,_Iquarter_4,_Iquarterto_1,_Iquarterto_10,_Iquarterto_11,_Iquarterto_12,_Iquarterto_13,_Iquarterto_14,_Iquarterto_15,_Iquarterto_2,_Iquarterto_3,_Iquarterto_4,_Iquarterto_5,_Iquarterto_6,_Iquarterto_7,_Iquarterto_8,_Iquarterto_9,_Iyear_1926,_Iyear_1927,_Iyear_1928,_Iyear_1929,_Iyear_1930,_Iyear_1931,_Iyear_1932,_Iyear_1933,_Iyear_1934,_Iyear_1935,_Iyear_1936,_Iyear_1937,_Iyear_1938,_Iyear_1939,_Iyear_1940,_Iyear_1941,_Iyear_1942,_Iyear_1943,_Iyear_1944,_Iyear_1945,_Iyear_1946,_Iyear_1947,_Iyear_1948,_Iyear_1949,_Iyear_1950,_Iyear_1951,_Iyear_1952,_Iyear_1953,_Iyear_1954,_Iyear_1955,_Iyear_1956,_Iyear_1957,_Iyear_1958,_Iyear_1959,_Iyear_1960,_Iyear_1961,_Iyear_1962,_Iyear_1963,_Iyear_1964,_Iyear_1965,_Iyear_1966,_Iyear_1967,_Iyear_1968,_Iyear_1969,_Iyear_1970,_Iyear_1971,_Iyear_1972,_Iyear_1973,_Iyear_1974,_Iyear_1975,_Iyear_1976,_Iyear_1977,_Iyear_1978,_Iyear_1979,_Iyear_1980,_Iyear_1981,_Iyear_1982,_Iyear_1983,_Iyear_1984,_Iyear_1985,_Iyear_1986,_Iyear_1987,_Iyear_1988,_Iyear_1989,_Iyear_1990,_Iyear_1991,_Iyear_1992,_Iyear_1993,_Iyear_1994,_Iyear_1995,_Iyear_1996,_Iyear_1997,_Iyear_1998,_Iyear_1999,_Iyear_2000,_Iyear_2001,_Iyear_2002,___char,___j,___l,a_groups,aba,abamin,abusedis,add1,adminrev,affirmdummy,afghan,age,age2,age2trun,age50,agego,agen_acq,ageon,alj,alst_code,altdisp,amicus,amicusapp,amicusresp,amon,ap_stid,appage,appbus,appel1,appel2,appfed,appfiduc,applfrom,appnatpr,appnonp,appres,appstate,appsubst,apptdemocrat,asame,asame_f1,asame_f2,asensame,asensame_f1,asensame_f2,assets,attyfee,ayear,ba,bank_ap1,bank_ap2,bank_r1,bank_r2,bapp,bast,boom,capric,caseload,casenum,casetyp1,casetyp2,citevol,city,civproc1,civproc2,classact,clinton,close,close2,close3,closerd,codej1,codej2,codej3,comment,conc0pro0merdummy,conc0promerdummy,conclint,concmer1,concmer2,concmerdummy,concpro0merdummy,concpro1,concpro2,concprodummy,concpromerdummy,concur,concurauthor,concurdummy,concurjoin,concurvalence,condole,confess,congresi,const1,const2,constit,counsel,counsel1,counsel2,count,courtesy,crmproc1,crmproc2,crossa,crossapp,crossl,csb,d,d12,d13,d23,dAds3,dF1Ads3,dF2Ads3,dL1Ads3,dL2Ads3,dL3Ads3,dL4Ads3,dL5Ads3,dak,dal,dar,day,daz,dca,dco,dcother,dct,dde,deathpen,decade,decade2,decuncon,democrat,denovo,dfl,dga,dhi,dhouse,did,dil,din,dio,direct1,direct2,discover,diss0pro0merdummy,diss0promerdummy,dissent,dissentdummy,dissentvalence,dissentvote,dissmer,dissmerdummy,disspro,disspro0merdummy,dissprodummy,disspromerdummy,distance,distjudg,district,districtjudgeinpanel,diverse,divided,divided1,dividedA,dks,dky,dla,dma,dmd,dme,dmi,dmn,dmo,dms,dmt,dnc,dnd,dne,dnh,dnj,dnm,dnv,dny,doh,dok,dor,dpa,dri,dropflag,dsc,dsd,dsen,dtn,dtx,dueproc,dummy,dut,dva,dvt,dwa,dwi,dwv,dwy,e1,e2,e3,e4,e5,e6,electoralpct,electoralpivotal,elevate,elevated,elevated2,endyear,entrap,erron,execord,exhaust,experience,experiencetrun,extra5,extreme,extremee,extremeleft,extremelefte,extremeright,extremerighte,extremescore,fartherd,fedlaw,fedlaw2,fedvst,female,foreign,fourties,freeinfo,frivapp,frivol,fromdistrictcourt,genapel1,genapel2,gender,geniss,geniss2,geniss3,genresp1,genresp2,genstand,graddeg1,graddeg2,graddeg3,gulf,habeas,hdem,hother,hrep,id,ids,immunity,improper,indict,indigent,inexperience,initiate,initiate1,initiate2,initiate3,initiate4,initiate5,injunct,insane,int_law,interven,isspty,j1appres,j1distjudge,j1party,j1sample,j1score,j2appres,j2distjudge,j2maj1,j2maj2,j2party,j2sample,j2score,j2vote1,j2vote2,j3appres,j3distjudge,j3maj1,j3maj2,j3party,j3sample,j3score,j3vote1,j3vote2,jdpp,judgdisc,judgecitations,judrev,juris,juryinst,keytotal,korean,last3,lastquarter,lastthreequarter,late,left,lengthopin,liable,liberaldissent,liberalvote,lik

In [54]:
## remove the columns that we are done with
newframe.drop(labels=['casenum','j2vote1','j2vote2','j2maj1','direct1',
                      'j2maj2','j3vote1','j3vote2','j3maj1','j3maj2','majvotes','ids'],axis=1,inplace=True)

In [55]:
new_cols=newframe.columns

In [56]:
new_cols=new_cols.tolist()

In [57]:
keep_cols=['j1score','j2score','j3score','popularpct','electoralpct','closerd','fartherd','dAds3','dF2Ads3',
           'dF1Ads3','dL1Ads3','dL2Ads3','dL3Ads3','dL4Ads3','dL5Ads3','logAds3','logL1Ads3','logL2Ads3','logF1Ads3',
          'logF2Ads3','decade2','propneg','likely_elev2','score','d12','d13','d23']

'''
This keep_cols are all float no categories. So null=0 will be fine.

for col in keep_cols:
    if len(pd.unique(newframe[col]))<4:
        print pd.unique(newframe[col]),col+"\n"
'''
#doubt that sth like dl4ads3 is also categories

'\nThis keep_cols are all float no categories. So null=0 will be fine.\n\nfor col in keep_cols:\n    if len(pd.unique(newframe[col]))<4:\n        print pd.unique(newframe[col]),col+"\n"\n'

In [58]:
for col in keep_cols:
    if col in new_cols:
        new_cols.remove(col)

In [59]:
#'dL1Ads3' in new_cols

False

In [60]:
newframe=pd.get_dummies(newframe,columns=new_cols,dummy_na=True)

In [61]:
newframe=newframe.fillna(value=0)         

In [63]:
newframe.to_csv('out.csv')